In [1]:
import numpy as np
import random
import sys
import io
import os
import glob
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt
import pandas as pd
from keras.layers import Dense, Input , Conv2D, ZeroPadding2D , Activation,Flatten
from sklearn.cross_validation import train_test_split
from keras.models import Model
import keras
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
%matplotlib inline


/home/nandwani_vaibhav/anaconda3/envs/fastai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/nandwani_vaibhav/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
IPython.display.Audio("train/audio/yes/004ae714_nohash_0.wav")

In [2]:
X_SIZE = 16000
IMG_SIZE = 128
def spectrogram(filepath):
	framerate, wav_data = wavfile.read(filepath)

	window_length = 512
	window_shift = 121

	if len(wav_data) > X_SIZE:
		wav_data = wav_data[:X_SIZE]

	X = np.zeros(X_SIZE).astype('float32')
	X[:len(wav_data)] += wav_data
	spec = np.zeros((IMG_SIZE, IMG_SIZE)).astype('float32')

	for i in range(IMG_SIZE):
		start = i * window_shift
		end = start + window_length
		sig = np.abs(np.fft.rfft(X[start:end] * np.hanning(window_length)))
		spec[:,i] = (sig[1:IMG_SIZE + 1])[::-1]

	spec = (spec-spec.min())/(spec.max()-spec.min())
	spec = np.log10((spec * 100 + 0.01))
	spec = (spec-spec.min())/(spec.max()-spec.min()) - 0.5

	return spec

In [10]:
x =  spectrogram("train/audio/yes/004ae714_nohash_0.wav")
x.shape

(128, 128)

In [ ]:
x=[]
y=[]
classes={}
folder=glob.glob("train/audio/*")
for n,file in enumerate(folder):
    
     for fname in glob.glob(file+"/*"):
        print(fname)
        x.append(spectrogram(fname))
        y.append(n)

train/audio/stop/3d53244b_nohash_3.wav
train/audio/stop/e8b6f6fe_nohash_0.wav
train/audio/stop/da8fa823_nohash_0.wav
train/audio/stop/24694eb6_nohash_0.wav
train/audio/stop/baeac2ba_nohash_0.wav


/home/nandwani_vaibhav/anaconda3/envs/fastai/lib/python3.6/site-packages/matplotlib/axes/_axes.py:7564: RuntimeWarning: divide by zero encountered in log10
  Z = 10. * np.log10(spec)


train/audio/stop/ced4e2a1_nohash_1.wav
train/audio/stop/f15a354c_nohash_1.wav
train/audio/stop/022cd682_nohash_0.wav
train/audio/stop/b5552931_nohash_1.wav
train/audio/stop/0cd323ec_nohash_0.wav
train/audio/stop/c661be6e_nohash_0.wav
train/audio/stop/fc94edb0_nohash_0.wav
train/audio/stop/9db2bfe9_nohash_0.wav
train/audio/stop/c1e0e8e3_nohash_4.wav
train/audio/stop/aef8dcf5_nohash_1.wav
train/audio/stop/8012c69d_nohash_3.wav
train/audio/stop/9c06a3f3_nohash_0.wav
train/audio/stop/88f3719f_nohash_0.wav
train/audio/stop/e0ff1e9c_nohash_0.wav
train/audio/stop/5af0ca83_nohash_0.wav
train/audio/stop/363c6bad_nohash_0.wav
train/audio/stop/86b89661_nohash_0.wav
train/audio/stop/48a9f771_nohash_0.wav
train/audio/stop/eb0676ec_nohash_3.wav
train/audio/stop/e41a903b_nohash_1.wav
train/audio/stop/de41f895_nohash_0.wav
train/audio/stop/8d5d076d_nohash_1.wav
train/audio/stop/6ceeb9aa_nohash_0.wav
train/audio/stop/187af8be_nohash_0.wav
train/audio/stop/49af4432_nohash_1.wav
train/audio/stop/c661be6e

train/audio/stop/2bd2cad5_nohash_0.wav
train/audio/stop/1890e5c3_nohash_0.wav
train/audio/stop/ffb86d3c_nohash_2.wav
train/audio/stop/94de6a6a_nohash_4.wav
train/audio/stop/26e573a9_nohash_1.wav
train/audio/stop/095847e4_nohash_0.wav
train/audio/stop/dff327c5_nohash_1.wav
train/audio/stop/4f2ab70c_nohash_1.wav
train/audio/stop/b93528e3_nohash_2.wav
train/audio/stop/3d53244b_nohash_1.wav
train/audio/stop/dbb40d24_nohash_2.wav
train/audio/stop/dea820ce_nohash_2.wav
train/audio/stop/74b73f88_nohash_0.wav
train/audio/stop/9aa21fa9_nohash_0.wav
train/audio/stop/b72e58c9_nohash_0.wav
train/audio/stop/6d2d3b0d_nohash_0.wav
train/audio/stop/89f3ab7d_nohash_0.wav
train/audio/stop/3c257192_nohash_3.wav
train/audio/stop/3aa6f4e2_nohash_1.wav
train/audio/stop/6078eb0d_nohash_0.wav
train/audio/stop/7846fd85_nohash_2.wav
train/audio/stop/1eddce1d_nohash_3.wav
train/audio/stop/2f666bb2_nohash_1.wav
train/audio/stop/facd97c0_nohash_1.wav
train/audio/stop/d5356b9a_nohash_0.wav
train/audio/stop/88f3719f

train/audio/stop/735845ab_nohash_2.wav
train/audio/stop/59c3a7f2_nohash_1.wav
train/audio/stop/1d919a90_nohash_1.wav
train/audio/stop/4634529e_nohash_0.wav
train/audio/stop/ad1429cf_nohash_1.wav
train/audio/stop/cb5d2c6e_nohash_0.wav
train/audio/stop/652b3da7_nohash_0.wav
train/audio/stop/5f8e50a0_nohash_0.wav
train/audio/stop/ea9f8e11_nohash_1.wav
train/audio/stop/b5552931_nohash_4.wav
train/audio/stop/62ef962d_nohash_0.wav
train/audio/stop/25132942_nohash_2.wav
train/audio/stop/b97c9f77_nohash_2.wav
train/audio/stop/a8688b67_nohash_0.wav
train/audio/stop/b4aa9fef_nohash_3.wav
train/audio/stop/67961766_nohash_0.wav
train/audio/stop/d1d3e64d_nohash_0.wav
train/audio/stop/0ab3b47d_nohash_0.wav
train/audio/stop/5c8af87a_nohash_3.wav
train/audio/stop/d197e3ae_nohash_2.wav
train/audio/stop/1dce06e8_nohash_1.wav
train/audio/stop/bdee441c_nohash_0.wav
train/audio/stop/0f7dc557_nohash_0.wav
train/audio/stop/5236848b_nohash_0.wav
train/audio/stop/918a2473_nohash_0.wav
train/audio/stop/acfd3bc3

...

train/audio/go/a1cff772_nohash_3.wav
train/audio/go/7303215d_nohash_1.wav
train/audio/go/1cec8d71_nohash_0.wav
train/audio/go/96a48d28_nohash_4.wav
train/audio/go/1fd85ee4_nohash_1.wav
train/audio/go/28ce0c58_nohash_6.wav
train/audio/go/611d2b50_nohash_3.wav
train/audio/go/da584bc0_nohash_0.wav
train/audio/go/3402e488_nohash_0.wav
train/audio/go/f19c1390_nohash_4.wav
train/audio/go/fe5c4a7a_nohash_0.wav
train/audio/go/15c563d7_nohash_4.wav
train/audio/go/53d5b86f_nohash_0.wav
train/audio/go/c948d727_nohash_1.wav
train/audio/go/553f1a79_nohash_1.wav
train/audio/go/91223b83_nohash_2.wav
train/audio/go/64220627_nohash_1.wav
train/audio/go/042186b8_nohash_0.wav
train/audio/go/439c84f4_nohash_1.wav
train/audio/go/dea820ce_nohash_0.wav
train/audio/go/a24cf51c_nohash_0.wav
train/audio/go/c79159aa_nohash_4.wav
train/audio/go/bbd0bbd0_nohash_3.wav
train/audio/go/9a4d12fd_nohash_0.wav
train/audio/go/d1a4fb3f_nohash_0.wav
train/audio/go/65040d9b_nohash_0.wav
train/audio/go/af405b69_nohash_0.wav
t

train/audio/go/964e8cfd_nohash_1.wav
train/audio/go/5fadb538_nohash_3.wav
train/audio/go/3fb8c268_nohash_0.wav
train/audio/go/69f609ec_nohash_1.wav
train/audio/go/8830e17f_nohash_1.wav
train/audio/go/4a294341_nohash_0.wav
train/audio/go/be91a00a_nohash_0.wav
train/audio/go/90b0b91a_nohash_0.wav
train/audio/go/726382d6_nohash_1.wav
train/audio/go/06a79a03_nohash_1.wav
train/audio/go/11a022ba_nohash_1.wav
train/audio/go/c205e625_nohash_1.wav
train/audio/go/9a3f97f8_nohash_0.wav
train/audio/go/39a6b995_nohash_0.wav
train/audio/go/e7ea8b76_nohash_5.wav
train/audio/go/7cbf645a_nohash_0.wav
train/audio/go/0d2bcf9d_nohash_0.wav
train/audio/go/9a69672b_nohash_4.wav
train/audio/go/56eab10e_nohash_1.wav
train/audio/go/50a8ef27_nohash_0.wav
train/audio/go/3903b558_nohash_0.wav
train/audio/go/e0344f60_nohash_0.wav
train/audio/go/ec74a8a5_nohash_0.wav
train/audio/go/333784b7_nohash_4.wav
train/audio/go/f953e1af_nohash_0.wav
train/audio/go/37dca74f_nohash_2.wav
train/audio/go/5eb5fc74_nohash_1.wav
t

train/audio/go/10ace7eb_nohash_4.wav
train/audio/go/7fa5fa1f_nohash_0.wav
train/audio/go/079d1020_nohash_0.wav
train/audio/go/dbb40d24_nohash_4.wav
train/audio/go/d91a159e_nohash_0.wav
train/audio/go/5e3dde6b_nohash_4.wav
train/audio/go/c4533c47_nohash_1.wav
train/audio/go/553f1a79_nohash_3.wav
train/audio/go/6272b231_nohash_0.wav
train/audio/go/1b4c9b89_nohash_0.wav
train/audio/go/a0f93943_nohash_2.wav
train/audio/go/62ff07ef_nohash_0.wav
train/audio/go/772db621_nohash_0.wav
train/audio/go/8a28231e_nohash_0.wav
train/audio/go/735845ab_nohash_3.wav
train/audio/go/32a19ecf_nohash_2.wav
train/audio/go/f9318c93_nohash_0.wav
train/audio/go/1bc45db9_nohash_0.wav
train/audio/go/26b28ea7_nohash_1.wav
train/audio/go/c0445658_nohash_3.wav
train/audio/go/f5733968_nohash_1.wav
train/audio/go/bbaa7946_nohash_0.wav
train/audio/go/2796ac50_nohash_0.wav
train/audio/go/a518d1cf_nohash_0.wav
train/audio/go/b5552931_nohash_3.wav
train/audio/go/b1114e4f_nohash_0.wav
train/audio/go/f3cee168_nohash_0.wav
t

train/audio/go/3c6d55b3_nohash_2.wav
train/audio/go/e1469561_nohash_1.wav
train/audio/go/c24d96eb_nohash_0.wav
train/audio/go/5ac04a92_nohash_0.wav
train/audio/go/8601f33b_nohash_0.wav
train/audio/go/fac74f6a_nohash_0.wav
train/audio/go/25040e85_nohash_1.wav
train/audio/go/e7ea8b76_nohash_4.wav
train/audio/go/0137b3f4_nohash_1.wav
train/audio/go/0d53e045_nohash_0.wav
train/audio/go/a879a2c3_nohash_2.wav
train/audio/go/de08912f_nohash_0.wav
train/audio/go/122c5aa7_nohash_3.wav
train/audio/go/5628d7b7_nohash_0.wav
train/audio/go/21e8c417_nohash_1.wav
train/audio/go/a709ddd3_nohash_1.wav
train/audio/go/b8872c20_nohash_1.wav
train/audio/go/e9901cf0_nohash_1.wav
train/audio/go/f9bdf10e_nohash_0.wav
train/audio/go/c4cfbe43_nohash_0.wav
train/audio/go/b8b8cb52_nohash_0.wav
train/audio/go/5705a0f9_nohash_0.wav
train/audio/go/8c7c9168_nohash_0.wav
train/audio/go/964e8cfd_nohash_4.wav
train/audio/go/c120e80e_nohash_5.wav
train/audio/go/69953f48_nohash_1.wav
train/audio/go/023a61ad_nohash_0.wav
t

train/audio/go/692a88e6_nohash_4.wav
train/audio/go/982babaf_nohash_0.wav
train/audio/go/743edf9d_nohash_3.wav
train/audio/go/f92e49f3_nohash_2.wav
train/audio/go/87728a88_nohash_0.wav
train/audio/go/a2b16113_nohash_0.wav
train/audio/go/5b09db89_nohash_2.wav
train/audio/go/edc53350_nohash_0.wav
train/audio/go/14872d06_nohash_2.wav
train/audio/go/686d030b_nohash_1.wav
train/audio/go/023a61ad_nohash_1.wav
train/audio/go/3c257192_nohash_2.wav
train/audio/go/0487ba9b_nohash_0.wav
train/audio/go/a1cff772_nohash_4.wav
train/audio/go/9f63152b_nohash_2.wav
train/audio/go/712e4d58_nohash_4.wav
train/audio/go/76b58705_nohash_1.wav
train/audio/go/7c1d8533_nohash_6.wav
train/audio/go/b52bd596_nohash_0.wav
train/audio/go/34ba417a_nohash_0.wav
train/audio/go/d90b4138_nohash_1.wav
train/audio/go/778a4a01_nohash_1.wav
train/audio/go/c2bc4489_nohash_0.wav
train/audio/go/012c8314_nohash_2.wav
train/audio/go/28ed6bc9_nohash_2.wav
train/audio/go/d103dd6e_nohash_0.wav
train/audio/go/da76aa58_nohash_1.wav
t

In [ ]:
x=np.array(x)
y=np.array(y)

#x=x.reshape((23682, 128, 128,1))

In [133]:
x=x.reshape((23682, 128, 128))

In [4]:
#len(x)
x.shape

(23682, 128, 128)

In [134]:
#np.save("data",x)
#np.save("labels",y)

In [3]:
x=np.load("data.npy")
y=np.load("labels.npy")

In [5]:
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.2,random_state=42)

In [6]:
ytrain = keras.utils.to_categorical(ytrain, 10)
ytest = keras.utils.to_categorical(ytest, 10)

In [58]:
def model(input_shape):
   

    X_input = Input(shape = input_shape)
    X = Conv1D(64, kernel_size=8, strides=2)(X_input)                                
    X = BatchNormalization()(X)                               
    X = Activation('relu')(X)                                 
    X = Dropout(0.6)(X)  
    print(X.shape)
    
    X = Conv1D(32, kernel_size=6, strides=2)(X)                                 
    X = BatchNormalization()(X)                                
    X = Activation('relu')(X)                                 
    X = Dropout(0.6)(X)  
    print(X.shape)
    
    X = Conv1D(16, kernel_size=6, strides=2)(X)                                 
    X = BatchNormalization()(X)                                 
    X = Activation('relu')(X)                                
    X = Dropout(0.6)(X)  
   
    print(X.shape) 
    
    X = GRU(units = 20, return_sequences = True)(X)   
    X = Dropout(0.8)(X)                                
    X = BatchNormalization()(X)                                  
    X = Dropout(0.6)(X)                                 
    
    print(X.shape)
    X=Flatten ()(X)
    X =Dense(10, activation = "softmax")(X)

   

 model = Model(inputs = X_input, outputs = X)
    
    return model

In [59]:


model=model(input_shape= (xtrain.shape[1:]))

(?, 61, 64)
(?, 28, 32)
(?, 12, 16)
(?, ?, 10)


In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 128, 128)          0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 61, 64)            65600     
_________________________________________________________________
batch_normalization_55 (Batc (None, 61, 64)            256       
_________________________________________________________________
activation_31 (Activation)   (None, 61, 64)            0         
_________________________________________________________________
dropout_67 (Dropout)         (None, 61, 64)            0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 28, 32)            12320     
_________________________________________________________________
batch_normalization_56 (Batc (None, 28, 32)            128       
__________

In [61]:
opt = Adam(lr=0.1, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [62]:
model.fit(xtrain, ytrain, batch_size = 5, epochs=2)

Epoch 1/2
18945/18945 [==============================] - 54s 3ms/step - loss: 0.3470 - acc: 0.8987
Epoch 2/2
18945/18945 [==============================] - 51s 3ms/step - loss: 0.3310 - acc: 0.9000


In [41]:
model.save("abc")

In [42]:
model.evaluate(xtest,ytest)

4737/4737 [==============================] - 3s 572us/step


[0.3253565415038523, 0.900000035750204]